In [1]:
import numpy as np
import qecstruct as qec

from decoding import IDENTITY, XOR_BULK, XOR_LEFT, XOR_RIGHT, SWAP
from decoding import ConstraintString, get_constraint_sites, apply_parity_constraints, get_codewords, prepare_codewords, apply_bias_channel, decode

from mdopt.mps.utils import create_simple_product_state, create_custom_product_state
from mdopt.utils.utils import mpo_to_matrix

In [ ]:
"""
In this experiment, we decode a classical linear error correcting code.
First, we build the MPS containing the superposition of all codewords.
Then, we demostrate simple decoding of a classical LDPC code using Dephasing DMRG --
our own built-in DMRG-like optimisation algorithm to solve the main component problem --
the problem of finding a computational basis state cotributing the most to a given state.
"""

In [ ]:
# Fixing a random seed
SEED = 123

tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]

# Defining the parameters of a classical LDPC code.
NUM_BITS, NUM_CHECKS = 10, 6
CHECK_DEGREE, BIT_DEGREE = 5, 3
if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
    raise ValueError("The Tanner graph of the code must be bipartite.")

# Constructing the code as a qecstruct object.
example_code = qec.random_regular_code(
    NUM_BITS, NUM_CHECKS, BIT_DEGREE, CHECK_DEGREE, qec.Rng(SEED)
)

# Preparing the initial state.
state = create_simple_product_state(NUM_BITS, which="+")
state_dense = state.dense(flatten=True)

# Getting the sites where each string of constraints should be applied.
code_constraint_sites = get_constraint_sites(example_code)

print("")
print("Checking the codeword superposition state:")
print("")

# Preparing the codeword superposition state by the MPS-MPO evolution.
state = apply_parity_constraints(state, code_constraint_sites, tensors)

# Preparing the codeword superposition state in the dense form.
for j in range(NUM_CHECKS):

    # Preparing the MPO.
    constraint_string = ConstraintString(tensors, code_constraint_sites[j])
    constraint_mpo = constraint_string.get_mpo()

    # Finding the starting site of the MPS to build a correct dense-form operator.
    START_SITE = min(constraint_string.flat())

    # Preparing the dense-form operator.
    identities_l = [IDENTITY for _ in range(START_SITE)]
    identities_r = [
        IDENTITY for _ in range(NUM_BITS - len(constraint_mpo) - START_SITE)
    ]
    full_mpo = identities_l + constraint_mpo + identities_r
    mpo_dense = mpo_to_matrix(full_mpo, interlace=False, group=True)

    # Doing the contraction in dense form.
    state_dense = mpo_dense @ state_dense

# Tolerance under which we round tensor elements to zero.
TOL = 1e-12
mps_dense = state.dense(flatten=True)
mps_dense[np.abs(mps_dense) < TOL] = 0

# Retreiving codewords.
cwords = get_codewords(example_code)
cwords_to_compare_mps = np.flatnonzero(mps_dense)
cwords_to_compare_dense = np.flatnonzero(state_dense)

print()
print("Codewords from the generator matrix:")
print(cwords)
print("Codewords from the dense-form simulation:")
print(cwords_to_compare_mps)
print("Codewords from the MPS-form simulation:")
print(cwords_to_compare_dense)
print("")
print(
    "All lists of codewords match:",
    np.logical_and(
        np.array_equal(cwords, cwords_to_compare_mps),
        np.array_equal(cwords_to_compare_mps, cwords_to_compare_dense),
    ),
)

In [ ]:
print("")
print("Retreiving a perturbed codeword:")
print("")

# Defining the parameters of a classical LDPC code.
NUM_BITS, NUM_CHECKS = 16, 12
CHECK_DEGREE, BIT_DEGREE = 4, 3
if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
    raise ValueError("The Tanner graph of the code must be bipartite.")

# Defining the bias channel parameter and the error probability.
PROB_ERROR = 0.15
PROB_CHANNEL = PROB_ERROR

# Maximum bond dimension for contractor/DMRG.
CHI_MAX_CONTRACTOR = 1e4
CHI_MAX_DMRG = 1e4
# Number of DMRG sweeps.
NUM_RUNS = 1

# Constructing the code as a qecstruct object.
example_code = qec.random_regular_code(
    NUM_BITS, NUM_CHECKS, BIT_DEGREE, CHECK_DEGREE, qec.Rng(SEED)
)

# Getting the sites where each string of constraints should be applied.
code_constraint_sites = get_constraint_sites(example_code)

# Building an initial and a perturbed codeword.
INITIAL_CODEWORD, PERTURBED_CODEWORD = prepare_codewords(
    example_code, PROB_ERROR, error_model=qec.BinarySymmetricChannel, seed=SEED
)
print("The initial codeword is", INITIAL_CODEWORD)
print("The perturbed codeword is", PERTURBED_CODEWORD)
print("")

# Building the corresponding matrix product states.
initial_codeword_state = create_custom_product_state(
    INITIAL_CODEWORD, form="Right-canonical"
)
perturbed_codeword_state = create_custom_product_state(
    PERTURBED_CODEWORD, form="Right-canonical"
)

# Passing the perturbed codeword state through the bias channel.
perturbed_codeword_state = apply_bias_channel(
    perturbed_codeword_state,
    codeword_string=PERTURBED_CODEWORD,
    prob_channel=PROB_CHANNEL,
)

print("Applying constraints:")
print("")
# Applying the parity constraints defined by the code.
perturbed_codeword_state = apply_parity_constraints(
    perturbed_codeword_state,
    code_constraint_sites,
    tensors,
    chi_max=CHI_MAX_CONTRACTOR,
    renormalise=True,
    strategy="naive",
    silent=False,
)

print("Decoding:")
print("")
# Decoding the perturbed codeword.
dmrg_container, success = decode(
    message=perturbed_codeword_state,
    codeword=initial_codeword_state,
    code=example_code,
    num_runs=NUM_RUNS,
    chi_max_dmrg=CHI_MAX_DMRG,
    cut=1e-10,
    silent=False,
)
print(
    "The overlap of the density MPO main component and the initial codeword state: ",
    success,
)
print(
    "__________________________________________________________________________________________"
)